In [1]:
import pandas as pd
import numpy as np

In [2]:
ssmm_candidates_1_11 = pd.read_csv('../library: main/ssmm_elections_candidates_1_11.csv', sep='\t', encoding='utf-8')
ssmm_candidates_1_11.head()


,conv_x_name,name,convocation_no,year_stream_no,academic_group_no,program_txt,announcement_link,announcement_dttm,votes_cnt,elected_flg,result_link,result_dttm,appearance_dttm
0,1_Левин_Андрей,Левин Андрей,1,1.1,101.0,Активный участник и организатор различных меро...,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,48.0,1,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00
1,1_Матвеев_Дмитрий_Александрович,Матвеев Дмитрий Александрович,1,1.1,101.0,NaN,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,49.0,1,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00
2,1_Власов_Артем_Андреевич,Власов Артем Андреевич,1,1.2,107.0,NaN,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,36.0,1,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00
3,1_Кириллова_Полина,Кириллова Полина,1,1.2,112.0,Мне нравится заниматься общественной деятельно...,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,40.0,1,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00
4,1_Семенов_Павел,Семенов Павел,1,1.2,112.0,"""Здравствуйте. Я так понимаю, что я уже поздно...",https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,20.0,0,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00


In [3]:
def define_classic_year_no(year_stream_no):
    if 'm' in year_stream_no:
        classic_year_no = year_stream_no
    else:
        classic_year_no = year_stream_no[0]
    return classic_year_no

# define_classic_year_no('2.1')

def define_specialization(stream_no):
    if stream_no in [1,2]:
        spec = 'Фундаментальная математика'
    elif stream_no == 3:
        spec = 'Фундаментальная механика'
    elif stream_no == 4:
        spec = 'Математика и экономическая теория' # старое: Математические методы экономики
    elif stream_no == 5:
        spec = 'Фундаментальная математика и математическая физика'
    elif stream_no == 0:
        spec = 'Магистратура'
    else:
        spec = 'ERROR'
    return spec

def define_short_specialization(stream_no):
    if stream_no in [1,2]:
        spec = 'Математика'
    elif stream_no == 3:
        spec = 'Механика'
    elif stream_no == 4:
        spec = 'Эконом' # старое: Математические методы экономики
    elif stream_no == 5:
        spec = 'ФММФ'
    elif stream_no == 0:
        spec = 'Магистратура'
    else:
        spec = 'ERROR'
    return spec

# С сентября 2021 года формируется группа 241 - экспериментальная группа программы (специализации) 
# "Фундаментальная математика и математическая физика".
# О программе можно посмотреть на сайте https://fmmp.math.msu.ru/about/

# учебные планы
# https://math.msu.ru/uplans


def define_course_level(year_stream_no):
    if 'm' in year_stream_no:
        course_level = 'Старшие курсы'
    elif int(year_stream_no[0]) <= 2:
        course_level = 'Младшие курсы'
    elif int(year_stream_no[0]) >= 3:
        course_level = 'Старшие курсы'
    else:
        course_level = 'ERROR'
    return course_level

def split_year_stream(year_stream_no):
    if 'm' in year_stream_no:
        year_no = int(year_stream_no[-1])
        stream_no = 0
    elif '.' in year_stream_no:
        year_no = int(year_stream_no[0])
        stream_no = int(year_stream_no[-1])
    else:
        year_no, stream_no = -1, -1
    return year_no, stream_no

# split_year_stream('m2')


In [4]:

ssmm_dict_year_stream = ssmm_candidates_1_11[['year_stream_no']].drop_duplicates().sort_values(['year_stream_no'],
              ascending = [True]).reset_index().drop(['index'], axis=1)
# ssmm_dict_year_stream['kek'] = ssmm_dict_year_stream['year_stream_no'].apply(lambda x: list(split_year_stream(x)))
ssmm_dict_year_stream['classic_year_no'] = ssmm_dict_year_stream['year_stream_no'].apply(lambda x: define_classic_year_no(x))
ssmm_dict_year_stream[['year_no','stream_no']] = pd.DataFrame(ssmm_dict_year_stream['year_stream_no'].apply(lambda x: list(split_year_stream(x))).tolist(), index=ssmm_dict_year_stream.index)
ssmm_dict_year_stream['course_level'] = ssmm_dict_year_stream['year_stream_no'].apply(lambda x: define_course_level(x))
ssmm_dict_year_stream['specialization'] = ssmm_dict_year_stream['stream_no'].apply(lambda x: define_specialization(x))
ssmm_dict_year_stream['short_specialization'] = ssmm_dict_year_stream['stream_no'].apply(lambda x: define_short_specialization(x))
ssmm_dict_year_stream

# df2[['team1','team2']] = pd.DataFrame(df2.teams.tolist(), index= df2.index)


,year_stream_no,classic_year_no,year_no,stream_no,course_level,specialization,short_specialization
0,1.1,1,1,1,Младшие курсы,Фундаментальная математика,Математика
1,1.2,1,1,2,Младшие курсы,Фундаментальная математика,Математика
2,1.3,1,1,3,Младшие курсы,Фундаментальная механика,Механика
3,2.1,2,2,1,Младшие курсы,Фундаментальная математика,Математика
4,2.2,2,2,2,Младшие курсы,Фундаментальная математика,Математика
5,2.3,2,2,3,Младшие курсы,Фундаментальная механика,Механика
6,2.5,2,2,5,Младшие курсы,Фундаментальная математика и математическая фи...,ФММФ
7,3.1,3,3,1,Старшие курсы,Фундаментальная математика,Математика
8,3.2,3,3,2,Старшие курсы,Фундаментальная математика,Математика
9,3.3,3,3,3,Старшие курсы,Фундаментальная механика,Механика


In [5]:
ssmm_dict_year_stream.to_csv('ssmm_dict_year_stream.csv', sep='\t', encoding='utf-8', index=False)
ssmm_dict_year_stream.to_csv('../library: main/ssmm_dict_year_stream.csv', sep='\t', encoding='utf-8', index=False)


In [6]:
def define_graduated_year(year_stream_no, year):
    if 'm' in year_stream_no:
        if year_stream_no == 'm2':
            graduated_flg = 1
        else:
            graduated_flg = 0
    elif int(year_stream_no[0]) == 5 and year < 2016:
        graduated_flg = 1
    elif int(year_stream_no[0]) == 6 and year >= 2016:
        graduated_flg = 1
    else:
        graduated_flg = 0
    return graduated_flg

def year_by_conv_no(conv_no):
    if conv_no >= 2:
        return conv_no+2010
    elif conv_no == 1:
        return 2012
    else:
        return -1


In [7]:
ssmm_dict_years = ssmm_candidates_1_11[['convocation_no','year_stream_no']].drop_duplicates().sort_values(['convocation_no','year_stream_no'],
              ascending = [True,True]).reset_index().drop(['index'], axis=1)
ssmm_dict_years['year'] = ssmm_dict_years['convocation_no'].apply(lambda x: year_by_conv_no(x) )
ssmm_dict_years['graduated_flg'] = ssmm_dict_years.apply(lambda x: define_graduated_year(x['year_stream_no'], x['year']), axis=1)

ssmm_dict_graduated_years = ssmm_dict_years
ssmm_dict_graduated_years


,convocation_no,year_stream_no,year,graduated_flg
0,1,1.1,2012,0
1,1,1.2,2012,0
2,1,1.3,2012,0
3,1,2.1,2012,0
4,1,2.2,2012,0
...,...,...,...,...
194,11,5.3,2021,0
195,11,5.4,2021,0
196,11,6.1,2021,1
197,11,6.3,2021,1


In [8]:
ssmm_dict_graduated_years.to_csv('ssmm_dict_graduated_years.csv', sep='\t', encoding='utf-8', index=False)
ssmm_dict_graduated_years.to_csv('../library: main/ssmm_dict_graduated_years.csv', sep='\t', encoding='utf-8', index=False)
